In [2]:
import pandas as pd

In [9]:
x = pd.read_csv('./data/2.remove_unbalance_missing_data/features_remove_missing_knn.csv')
#x.drop(['Unnamed: 0'], axis = 1, inplace = True)
x.shape

(1700, 104)

# 1. binary column중 unbalance한 Feature 삭제 therehold = 0.01

In [10]:
binary_column_info = {}

cnt = 0

for column in x.columns:
    # 유일한 값들 확인
    unique_values = x[column].dropna().unique()
    
    # 0과 1만 있는지 확인
    if set(unique_values) <= {0, 1}:
        count_0 = (x[column] == 0).sum()
        count_1 = (x[column] == 1).sum()
        total_count = count_0 + count_1
        percent_0 = (count_0 / total_count) * 100
        percent_1 = (count_1 / total_count) * 100
        binary_column_info[column] = {
            '0 count': count_0, 
            '1 count': count_1,
            '0 percent': percent_0,
            '1 percent': percent_1
        }

li = []
for column, counts in binary_column_info.items():

    if(counts['1 percent'] <= 1):
        print(column, "is unbalanced.", counts['0 count'], counts['1 count'])
        cnt +=1
        li.append(column)
    else:
        print(column, "is balanced.", counts['0 count'], counts['1 count'])

print(cnt)

SEX is balanced. 635 1065
SIM_GIPERT is balanced. 1643 57
nr_11 is balanced. 1658 42
nr_01 is unbalanced. 1696 4
nr_02 is balanced. 1681 19
nr_03 is balanced. 1665 35
nr_04 is balanced. 1671 29
nr_07 is unbalanced. 1699 1
nr_08 is unbalanced. 1696 4
np_01 is unbalanced. 1698 2
np_04 is unbalanced. 1697 3
np_05 is unbalanced. 1689 11
np_07 is unbalanced. 1699 1
np_08 is unbalanced. 1694 6
np_09 is unbalanced. 1698 2
np_10 is unbalanced. 1697 3
endocr_01 is balanced. 1472 228
endocr_02 is balanced. 1658 42
endocr_03 is unbalanced. 1687 13
zab_leg_01 is balanced. 1566 134
zab_leg_02 is balanced. 1579 121
zab_leg_03 is balanced. 1663 37
zab_leg_04 is unbalanced. 1691 9
zab_leg_06 is balanced. 1678 22
O_L_POST is balanced. 1590 110
K_SH_POST is balanced. 1653 47
MP_TP_POST is balanced. 1586 114
SVT_POST is unbalanced. 1692 8
GT_POST is unbalanced. 1692 8
FIB_G_POST is unbalanced. 1685 15
IM_PG_P is balanced. 1650 50
ritm_ecg_p_01 is balanced. 555 1145
ritm_ecg_p_02 is balanced. 1605 95
ritm

In [11]:
x = x.drop(columns=li)
x.shape

(1700, 73)

In [12]:
x.to_csv('./data/2.remove_unbalance_missing_data/1_1.csv')

# 2. VAE Test

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the data
file_path = './1_1.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the data
print(data.head())

# Preprocess the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data.drop(columns=['Unnamed: 0']))

# Split the data into training and validation sets
x_train, x_val = train_test_split(data_scaled, test_size=0.2, random_state=42)

# Define the encoder
input_dim = x_train.shape[1]
latent_dim = 2  # You can adjust the dimensionality of the latent space

inputs = layers.Input(shape=(input_dim,))
h = layers.Dense(64, activation='relu')(inputs)
h = layers.Dense(32, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_var = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder
decoder_input = layers.Input(shape=(latent_dim,))
decoder_h = layers.Dense(32, activation='relu')
decoder_h2 = layers.Dense(64, activation='relu')
decoder_mean = layers.Dense(input_dim, activation='sigmoid')

_h_decoded = decoder_h(decoder_input)
_h_decoded = decoder_h2(_h_decoded)
_outputs = decoder_mean(_h_decoded)

decoder = models.Model(decoder_input, _outputs)

# Define the VAE model
h_decoded = decoder_h(z)
h_decoded = decoder_h2(h_decoded)
outputs = decoder_mean(h_decoded)
vae = models.Model(inputs, outputs)

# Define the loss
reconstruction_loss = tf.keras.losses.mse(inputs, outputs)
reconstruction_loss *= input_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Train the VAE model
vae.fit(x_train, epochs=50, batch_size=32, validation_data=(x_val, None))

# Generate new samples using the trained VAE
def generate_samples(decoder, num_samples, latent_dim):
    z_sample = np.random.normal(size=(num_samples, latent_dim))
    generated_data = decoder.predict(z_sample)
    return scaler.inverse_transform(generated_data)

# Generate 100 new samples
new_samples = generate_samples(decoder, 10, latent_dim)

# Convert generated samples to DataFrame and round all values to integers
new_samples_df = pd.DataFrame(new_samples, columns=data.columns[1:])
#new_samples_df = new_samples_df.round(1)

# Display the generated samples
print(new_samples_df.head())


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [62]:
#값 분포 확인
for column in new_samples_df.columns:
    unique_values = new_samples_df[column].dropna().unique()
    print(column, unique_values)

AGE [61.862152 61.904064 61.873535 61.8622   61.88735  61.86219  62.29251
 61.872353 61.862274 61.862232]
SEX [0.62647057 0.6264798  0.6264786  0.6264807  0.6273904  0.6264753 ]
INF_ANAM [0.5554382  0.56035745 0.56237835 0.5555472  0.5624952  0.555519
 0.60098267 0.5607117  0.5557131  0.5556157 ]
IBS_POST [1.1676898 1.177472  1.1714579 1.1677295 1.1740502 1.1677223 1.2279601
 1.1714482 1.1677824 1.1677508]
SIM_GIPERT [0.03352957 0.0335954  0.03359849 0.0335298  0.0336033  0.03352977
 0.03554852 0.03359427 0.03353011 0.03352992]
ZSN_A [0.19178294 0.19483143 0.19401135 0.19180085 0.19538674 0.19179365
 0.21972127 0.19313085 0.19183792 0.19181687]
nr_11 [0.02470591 0.02473683 0.02472922 0.02470594 0.0247322  0.02470592
 0.02584688 0.02472484 0.02470601 0.02470597]
nr_02 [0.01117648 0.01119135 0.01118796 0.01117649 0.01119956 0.01117649
 0.01187023 0.01118534 0.01117652 0.0111765 ]
nr_03 [0.02058899 0.02081238 0.02075322 0.02058991 0.02079522 0.02058982
 0.0244072  0.02075441 0.02059102 0.

In [17]:
combined_df = pd.concat([data, new_samples_df], ignore_index=True)
combined_df = combined_df.drop(columns=['Unnamed: 0'])
combined_df.to_csv('1_2.csv')

# 3. ADASYN

In [15]:
from imblearn.over_sampling import ADASYN
import pandas as pd
from collections import Counter

x = pd.read_csv('./data/2.remove_unbalance_missing_data/1_1.csv')
y = pd.read_csv('./data/0.origin_data/target.csv')
y = y['LET_IS']

sampling_strategy = {label: count * 2 for label, count in Counter(y).items()}
ada = ADASYN(random_state=42, sampling_strategy=sampling_strategy)
X_res, y_res = ada.fit_resample(x, y)

print(f'Original dataset shape {x.shape, y.shape}')
print(f'Resampled dataset shape {X_res.shape, y_res.shape}')


Original dataset shape ((1700, 74), (1700,))
Resampled dataset shape ((3388, 74), (3388,))


In [16]:
#값 분포 확인

binary_column_info = {}
cnt = 0
for column in X_res.columns:
    unique_values = X_res[column].dropna().unique()
    if set(unique_values) <= {0, 1}:
        count_0 = (X_res[column] == 0).sum()
        count_1 = (X_res[column] == 1).sum()
        total_count = count_0 + count_1
        percent_0 = (count_0 / total_count) * 100
        percent_1 = (count_1 / total_count) * 100
        binary_column_info[column] = {
            '0 count': count_0, 
            '1 count': count_1,
            '0 percent': percent_0,
            '1 percent': percent_1
        }
li = []
for column, counts in binary_column_info.items():

    if(counts['1 percent'] <= 1):
        print(column, "is unbalanced.", counts['0 count'], counts['1 count'])
        cnt +=1
        li.append(column)
    else:
        print(column, "is balanced.", counts['0 count'], counts['1 count'])

print(cnt)

SEX is balanced. 1973 1415
SIM_GIPERT is balanced. 3331 57
nr_11 is balanced. 3346 42
nr_02 is unbalanced. 3369 19
nr_03 is balanced. 3353 35
nr_04 is unbalanced. 3356 32
endocr_01 is balanced. 3076 312
endocr_02 is balanced. 3346 42
zab_leg_01 is balanced. 3251 137
zab_leg_02 is balanced. 3204 184
zab_leg_03 is balanced. 3350 38
zab_leg_06 is unbalanced. 3366 22
O_L_POST is balanced. 3271 117
K_SH_POST is balanced. 3336 52
MP_TP_POST is balanced. 3265 123
IM_PG_P is balanced. 3337 51
ritm_ecg_p_01 is balanced. 1731 1657
ritm_ecg_p_02 is balanced. 3288 100
ritm_ecg_p_04 is unbalanced. 3365 23
ritm_ecg_p_07 is balanced. 2945 443
ritm_ecg_p_08 is balanced. 3342 46
n_r_ecg_p_01 is balanced. 3329 59
n_r_ecg_p_03 is balanced. 3177 211
n_r_ecg_p_04 is balanced. 3317 71
n_r_ecg_p_05 is balanced. 3316 72
n_r_ecg_p_06 is balanced. 3354 34
n_p_ecg_p_03 is unbalanced. 3356 32
n_p_ecg_p_06 is unbalanced. 3360 28
n_p_ecg_p_07 is balanced. 3283 105
n_p_ecg_p_10 is balanced. 3354 34
n_p_ecg_p_11 is u

In [17]:
X_res['AGE'] = X_res['AGE'].round().astype(int)
X_res.to_csv('./data/2.remove_unbalance_missing_data/1_3.csv')
y_res.to_csv('./data/2.remove_unbalance_missing_data/1_3y.csv')

# 4. SMOTE

In [18]:
from imblearn.over_sampling import SMOTE
from collections import Counter

x = pd.read_csv('./data/2.remove_unbalance_missing_data/1_1.csv')
y = pd.read_csv('./data/0.origin_data/target.csv')
y = y['LET_IS']

# 샘플링 전략 정의
sampling_strategy = {label: count * 2 for label, count in Counter(y).items()}

# SMOTE 적용
smote = SMOTE(random_state=42, sampling_strategy=sampling_strategy)
X_res, y_res = smote.fit_resample(x, y)

# 원본 및 리샘플링된 데이터셋의 형태 출력
print(f'Original dataset shape {x.shape, y.shape}')
print(f'Resampled dataset shape {X_res.shape, y_res.shape}')

Original dataset shape ((1700, 74), (1700,))
Resampled dataset shape ((3400, 74), (3400,))


In [19]:
#값 분포 확인
binary_column_info = {}
cnt = 0
for column in X_res.columns:
    unique_values = X_res[column].dropna().unique()
    if set(unique_values) <= {0, 1}:
        count_0 = (X_res[column] == 0).sum()
        count_1 = (X_res[column] == 1).sum()
        total_count = count_0 + count_1
        percent_0 = (count_0 / total_count) * 100
        percent_1 = (count_1 / total_count) * 100
        binary_column_info[column] = {
            '0 count': count_0, 
            '1 count': count_1,
            '0 percent': percent_0,
            '1 percent': percent_1
        }
li = []
for column, counts in binary_column_info.items():

    if(counts['1 percent'] <= 1):
        print(column, "is unbalanced.", counts['0 count'], counts['1 count'])
        cnt +=1
        li.append(column)
    else:
        print(column, "is balanced.", counts['0 count'], counts['1 count'])

print(cnt)

SEX is balanced. 1619 1781
SIM_GIPERT is balanced. 3340 60
nr_11 is balanced. 3358 42
nr_02 is unbalanced. 3381 19
nr_03 is balanced. 3365 35
nr_04 is unbalanced. 3370 30
endocr_01 is balanced. 3133 267
endocr_02 is balanced. 3358 42
zab_leg_01 is balanced. 3247 153
zab_leg_02 is balanced. 3264 136
zab_leg_03 is balanced. 3360 40
zab_leg_06 is unbalanced. 3377 23
O_L_POST is balanced. 3277 123
K_SH_POST is balanced. 3326 74
MP_TP_POST is balanced. 3277 123
IM_PG_P is balanced. 3350 50
ritm_ecg_p_01 is balanced. 1446 1954
ritm_ecg_p_02 is balanced. 3297 103
ritm_ecg_p_04 is unbalanced. 3377 23
ritm_ecg_p_07 is balanced. 2952 448
ritm_ecg_p_08 is balanced. 3354 46
n_r_ecg_p_01 is balanced. 3338 62
n_r_ecg_p_03 is balanced. 3175 225
n_r_ecg_p_04 is balanced. 3326 74
n_r_ecg_p_05 is balanced. 3325 75
n_r_ecg_p_06 is unbalanced. 3367 33
n_p_ecg_p_03 is unbalanced. 3366 34
n_p_ecg_p_06 is unbalanced. 3372 28
n_p_ecg_p_07 is balanced. 3292 108
n_p_ecg_p_10 is unbalanced. 3366 34
n_p_ecg_p_11 

In [20]:
X_res['AGE'] = X_res['AGE'].round().astype(int)
X_res.tail()
X_res.to_csv('./data/2.remove_unbalance_missing_data/1_4.csv')
y_res.to_csv('./data/2.remove_unbalance_missing_data/1_4y.csv')

# 5. Border-SMOTE

In [22]:
import pandas as pd
from imblearn.over_sampling import BorderlineSMOTE
from collections import Counter

# 데이터 로드
x = pd.read_csv('./data/2.remove_unbalance_missing_data/1_1.csv')
y = pd.read_csv('./data/0.origin_data/target.csv')
y = y['LET_IS']

# 샘플링 전략 정의
sampling_strategy = {label: count * 2 for label, count in Counter(y).items()}

# Borderline SMOTE 적용
borderline_smote = BorderlineSMOTE(random_state=42, sampling_strategy=sampling_strategy)
X_res, y_res = borderline_smote.fit_resample(x, y)

# 원본 및 리샘플링된 데이터셋의 형태 출력
print(f'Original dataset shape {x.shape, y.shape}')
print(f'Resampled dataset shape {X_res.shape, y_res.shape}')

Original dataset shape ((1700, 74), (1700,))
Resampled dataset shape ((3400, 74), (3400,))


In [23]:
#값 분포 확인
binary_column_info = {}
cnt = 0
for column in X_res.columns:
    unique_values = X_res[column].dropna().unique()
    if set(unique_values) <= {0, 1}:
        count_0 = (X_res[column] == 0).sum()
        count_1 = (X_res[column] == 1).sum()
        total_count = count_0 + count_1
        percent_0 = (count_0 / total_count) * 100
        percent_1 = (count_1 / total_count) * 100
        binary_column_info[column] = {
            '0 count': count_0, 
            '1 count': count_1,
            '0 percent': percent_0,
            '1 percent': percent_1
        }
li = []
for column, counts in binary_column_info.items():

    if(counts['1 percent'] <= 1):
        print(column, "is unbalanced.", counts['0 count'], counts['1 count'])
        cnt +=1
        li.append(column)
    else:
        print(column, "is balanced.", counts['0 count'], counts['1 count'])

print(cnt)

SEX is balanced. 2119 1281
SIM_GIPERT is balanced. 3342 58
nr_11 is balanced. 3358 42
nr_02 is unbalanced. 3381 19
nr_03 is balanced. 3365 35
nr_04 is unbalanced. 3368 32
endocr_01 is balanced. 3022 378
endocr_02 is balanced. 3358 42
zab_leg_01 is balanced. 3262 138
zab_leg_02 is balanced. 3274 126
zab_leg_03 is balanced. 3362 38
zab_leg_06 is unbalanced. 3378 22
O_L_POST is balanced. 3285 115
K_SH_POST is balanced. 3348 52
MP_TP_POST is balanced. 3280 120
IM_PG_P is balanced. 3347 53
ritm_ecg_p_01 is balanced. 1502 1898
ritm_ecg_p_02 is balanced. 3298 102
ritm_ecg_p_04 is unbalanced. 3377 23
ritm_ecg_p_07 is balanced. 2998 402
ritm_ecg_p_08 is balanced. 3354 46
n_r_ecg_p_01 is balanced. 3341 59
n_r_ecg_p_03 is balanced. 3190 210
n_r_ecg_p_04 is balanced. 3331 69
n_r_ecg_p_05 is balanced. 3326 74
n_r_ecg_p_06 is unbalanced. 3368 32
n_p_ecg_p_03 is unbalanced. 3368 32
n_p_ecg_p_06 is unbalanced. 3372 28
n_p_ecg_p_07 is balanced. 3294 106
n_p_ecg_p_10 is unbalanced. 3366 34
n_p_ecg_p_11 

In [24]:
X_res['AGE'] = X_res['AGE'].round().astype(int)
X_res.tail()
X_res.to_csv('./data/2.remove_unbalance_missing_data/1_5.csv')
y_res.to_csv('./data/2.remove_unbalance_missing_data/1_5y.csv')

# 6. SVMSMOTE

In [25]:
import pandas as pd
from imblearn.over_sampling import SVMSMOTE
from collections import Counter

# 데이터 로드
x = pd.read_csv('./data/2.remove_unbalance_missing_data/1_1.csv')
y = pd.read_csv('./data/0.origin_data/target.csv')
y = y['LET_IS']

# 증강 전략 설정: 원본 데이터의 2배로 증강
sampling_strategy = {label: count * 2 for label, count in Counter(y).items()}

# SVMSMOTE 적용
svm_smote = SVMSMOTE(random_state=42, sampling_strategy=sampling_strategy)
X_res, y_res = svm_smote.fit_resample(x, y)

print(f'Original dataset shape {x.shape, y.shape}')
print(f'Resampled dataset shape {X_res.shape, y_res.shape}')

Original dataset shape ((1700, 74), (1700,))
Resampled dataset shape ((3353, 74), (3353,))


In [26]:
#값 분포 확인
binary_column_info = {}
cnt = 0
for column in X_res.columns:
    unique_values = X_res[column].dropna().unique()
    if set(unique_values) <= {0, 1}:
        count_0 = (X_res[column] == 0).sum()
        count_1 = (X_res[column] == 1).sum()
        total_count = count_0 + count_1
        percent_0 = (count_0 / total_count) * 100
        percent_1 = (count_1 / total_count) * 100
        binary_column_info[column] = {
            '0 count': count_0, 
            '1 count': count_1,
            '0 percent': percent_0,
            '1 percent': percent_1
        }
li = []
for column, counts in binary_column_info.items():

    if(counts['1 percent'] <= 1):
        print(column, "is unbalanced.", counts['0 count'], counts['1 count'])
        cnt +=1
        li.append(column)
    else:
        print(column, "is balanced.", counts['0 count'], counts['1 count'])

print(cnt)

SEX is balanced. 1824 1529
SIM_GIPERT is balanced. 3263 90
nr_11 is balanced. 3300 53
nr_02 is unbalanced. 3332 21
nr_03 is balanced. 3318 35
nr_04 is unbalanced. 3322 31
endocr_01 is balanced. 2917 436
endocr_02 is balanced. 3278 75
zab_leg_01 is balanced. 3208 145
zab_leg_02 is balanced. 3172 181
zab_leg_03 is balanced. 3292 61
zab_leg_06 is balanced. 3317 36
O_L_POST is balanced. 3234 119
K_SH_POST is balanced. 3254 99
MP_TP_POST is balanced. 3222 131
IM_PG_P is balanced. 3276 77
ritm_ecg_p_01 is balanced. 1234 2119
ritm_ecg_p_02 is balanced. 3226 127
ritm_ecg_p_04 is balanced. 3307 46
ritm_ecg_p_07 is balanced. 2885 468
ritm_ecg_p_08 is balanced. 3281 72
n_r_ecg_p_01 is balanced. 3287 66
n_r_ecg_p_03 is balanced. 3139 214
n_r_ecg_p_04 is balanced. 3278 75
n_r_ecg_p_05 is balanced. 3254 99
n_r_ecg_p_06 is unbalanced. 3320 33
n_p_ecg_p_03 is balanced. 3293 60
n_p_ecg_p_06 is balanced. 3301 52
n_p_ecg_p_07 is balanced. 3201 152
n_p_ecg_p_10 is balanced. 3316 37
n_p_ecg_p_11 is balance

In [27]:
X_res['AGE'] = X_res['AGE'].round().astype(int)
X_res.tail()
X_res.to_csv('./data/2.remove_unbalance_missing_data/1_6.csv')
y_res.to_csv('./data/2.remove_unbalance_missing_data/1_6y.csv')